# Task Navigation Efficiency Evaluator

## Objective
This sample demonstrates how to use task navigation efficiency evaluator on agent data. The supported input formats include:
- simple data such as strings and `dict` describing task responses;
- user-agent conversations in the form of list of agent messages. 

## Time

You should expect to spend about 20 minutes running this notebook. 

## Before you begin
This is a deterministic evaluator that compares navigation paths.

### Prerequisite
```bash
pip install azure-ai-projects azure-identity openai
```
Set these environment variables with your own values:
1) **AZURE_AI_PROJECT_ENDPOINT** - Your Azure AI project endpoint in format: `https://<account_name>.services.ai.azure.com/api/projects/<project_name>`
2) **AZURE_AI_MODEL_DEPLOYMENT_NAME** - The deployment name of the model for this AI-assisted evaluator (e.g., gpt-4o-mini)

The Task Navigation Efficiency Evaluator measures how efficiently an agent navigates through a sequence of actions compared to an optimal task completion path.

The evaluator provides comprehensive evaluation with both binary matching results and additional detailed P\R\F1 results:

**Primary Result:**
- **Binary Match Result**: Pass/Fail based on the selected matching mode

**Available Matching Modes:**
- **Exact Match**: Agent's tool calls must exactly match the ground truth (default)
- **In-Order Match**: All ground truth steps must appear in correct order (allows extra steps)
- **Any-Order Match**: All ground truth steps must appear with sufficient frequency (most lenient)

**Properties Bag Additional Metrics (0.0 - 1.0):**
- **Precision**: How many of the agent's steps were necessary (relevant to ground truth)
- **Recall**: How many of the required steps were executed by the agent  
- **F1 Score**: Harmonic mean of precision and recall

The evaluation requires the following inputs:
- **Response**: The agent's response containing tool calls as a list of messages or string
- **Ground Truth**: List of expected tool/action steps as strings, or tuple with parameters for matching

### Initialize Task Navigation Efficiency Evaluator


In [ ]:
import os
from openai.types.evals.create_eval_jsonl_run_data_source_param import SourceFileContentContent
from pprint import pprint
from agent_utils import run_evaluator

# Get environment variables
deployment_name = os.environ["AZURE_AI_MODEL_DEPLOYMENT_NAME"]

# Data source configuration (defines the schema for evaluation inputs)
data_source_config = {
    "type": "custom",
    "item_schema": {
        "type": "object",
        "properties": {
            "response": {
                "type": "array"
            },
            "ground_truth": {
                "type": "array"
            }
        },
        "required": ["response", "ground_truth"]
    },
    "include_sample_schema": True
}

# Data mapping (maps evaluation inputs to evaluator parameters)
data_mapping = {
    "response": "{{item.response}}",
    "ground_truth": "{{item.ground_truth}}"
}

# Initialization parameters for the evaluator
initialization_parameters = {
    "matching_mode": "exact_match"  # Can be "exact_match", "in_order_match", or "any_order_match"
}

# Initialize the evaluation_contents list - we'll append all test cases here
evaluation_contents = []

### Samples

#### Sample 1: Perfect Path (Exact Match)

In [ ]:
# Test Case 1: Perfect Path (Exact Match) - Agent follows the exact optimal path
response_perfect = [
    {
        "role": "assistant",
        "content": [{"type": "tool_call", "tool_call_id": "call_1", "name": "search", "arguments": {}}],
    },
    {
        "role": "assistant", 
        "content": [{"type": "tool_call", "tool_call_id": "call_2", "name": "analyze", "arguments": {}}],
    },
    {
        "role": "assistant",
        "content": [{"type": "tool_call", "tool_call_id": "call_3", "name": "report", "arguments": {}}],
    },
]

ground_truth_perfect = ["search", "analyze", "report"]

# Append to evaluation_contents
evaluation_contents.append(
    SourceFileContentContent(
        item={
            "response": response_perfect,
            "ground_truth": ground_truth_perfect
        }
    )
)

#### Sample 2: Efficient Path with Extra Steps

In [ ]:
# Test Case 2: Path with Extra Steps - Agent performs all required steps but with extra unnecessary step
response_extra = [
    {
        "role": "assistant",
        "content": [{"type": "tool_call", "tool_call_id": "call_1", "name": "search", "arguments": {}}],
    },
    {
        "role": "assistant",
        "content": [{"type": "tool_call", "tool_call_id": "call_2", "name": "validate", "arguments": {}}],
    },
    {
        "role": "assistant", 
        "content": [{"type": "tool_call", "tool_call_id": "call_3", "name": "analyze", "arguments": {}}],
    },
    {
        "role": "assistant",
        "content": [{"type": "tool_call", "tool_call_id": "call_4", "name": "report", "arguments": {}}],
    },
]

ground_truth_extra = ["search", "analyze", "report"]

# Append to evaluation_contents
evaluation_contents.append(
    SourceFileContentContent(
        item={
            "response": response_extra,
            "ground_truth": ground_truth_extra
        }
    )
)

#### Sample 3: Inefficient Path (Wrong Order)

In [ ]:
# Test Case 3: Wrong Order - Agent performs all required steps but in wrong order
response_wrong_order = [
    {
        "role": "assistant",
        "content": [{"type": "tool_call", "tool_call_id": "call_1", "name": "report", "arguments": {}}],
    },
    {
        "role": "assistant",
        "content": [{"type": "tool_call", "tool_call_id": "call_2", "name": "search", "arguments": {}}],
    },
    {
        "role": "assistant", 
        "content": [{"type": "tool_call", "tool_call_id": "call_3", "name": "analyze", "arguments": {}}],
    },
]

ground_truth_wrong_order = ["search", "analyze", "report"]

# Append to evaluation_contents
evaluation_contents.append(
    SourceFileContentContent(
        item={
            "response": response_wrong_order,
            "ground_truth": ground_truth_wrong_order
        }
    )
)

#### Sample 4: Incomplete Path (Missing Steps)

In [ ]:
# Test Case 4: Missing Steps - Agent performs only some of the required steps (incomplete)
response_missing = [
    {
        "role": "assistant",
        "content": [{"type": "tool_call", "tool_call_id": "call_1", "name": "search", "arguments": {}}],
    },
    {
        "role": "assistant",
        "content": [{"type": "tool_call", "tool_call_id": "call_2", "name": "analyze", "arguments": {}}],
    },
]

ground_truth_missing = ["search", "analyze", "report"]

# Append to evaluation_contents
evaluation_contents.append(
    SourceFileContentContent(
        item={
            "response": response_missing,
            "ground_truth": ground_truth_missing
        }
    )
)

#### Sample 5: Real-World Customer Service Scenario

In [ ]:
# Test Case 5: Customer Service Scenario - Real-world example: Customer service agent handling a refund request
response_service = [
    {
        "role": "assistant",
        "content": [{"type": "tool_call", "tool_call_id": "call_1", "name": "lookup_order", "arguments": {"order_id": "12345"}}],
    },
    {
        "role": "assistant",
        "content": [{"type": "tool_call", "tool_call_id": "call_2", "name": "check_inventory", "arguments": {"product_id": "ABC123"}}],
    },
    {
        "role": "assistant", 
        "content": [{"type": "tool_call", "tool_call_id": "call_3", "name": "calculate_refund", "arguments": {"order_id": "12345"}}],
    },
    {
        "role": "assistant",
        "content": [{"type": "tool_call", "tool_call_id": "call_4", "name": "process_refund", "arguments": {"order_id": "12345", "amount": "29.99"}}],
    },
]

ground_truth_service = ["lookup_order", "check_inventory", "calculate_refund", "process_refund"]

# Append to evaluation_contents
evaluation_contents.append(
    SourceFileContentContent(
        item={
            "response": response_service,
            "ground_truth": ground_truth_service
        }
    )
)

#### Sample 6: Complex Path with Duplicates

In [ ]:
# Test Case 6: Complex Path with Duplicates - Agent repeats some steps and includes extra ones
response_duplicates = [
    {
        "role": "assistant",
        "content": [{"type": "tool_call", "tool_call_id": "call_1", "name": "search", "arguments": {}}],
    },
    {
        "role": "assistant",
        "content": [{"type": "tool_call", "tool_call_id": "call_2", "name": "search", "arguments": {}}],  # duplicate
    },
    {
        "role": "assistant",
        "content": [{"type": "tool_call", "tool_call_id": "call_4", "name": "analyze", "arguments": {}}],
    },
    {
        "role": "assistant",
        "content": [{"type": "tool_call", "tool_call_id": "call_5", "name": "report", "arguments": {}}],
    },
]

ground_truth_duplicates = ["search", "analyze", "report"]

# Append to evaluation_contents
evaluation_contents.append(
    SourceFileContentContent(
        item={
            "response": response_duplicates,
            "ground_truth": ground_truth_duplicates
        }
    )
)

#### Sample 7: Tuple Format with Parameters

In [ ]:
# Test Case 7: Tuple Format with Parameters - Supports exact parameter matching
response_params = [
    {
        "role": "assistant",
        "content": [{"type": "tool_call", "tool_call_id": "call_1", "name": "search", "arguments": {"query": "test"}}],
    },
]

# Ground truth using tuple format: (tool_names, parameters_dict)
# Parameters must match exactly for tools to be considered matching
ground_truth_params = [["search"], {"search": {"query": "test"}}]

# Append to evaluation_contents
evaluation_contents.append(
    SourceFileContentContent(
        item={
            "response": response_params,
            "ground_truth": ground_truth_params
        }
    )
)

### Execute Batch Evaluation
Run all test cases together using the batch evaluation approach.

In [ ]:
results = run_evaluator(
    evaluator_name="task_navigation_efficiency",
    evaluation_contents=evaluation_contents,
    data_source_config=data_source_config,
    initialization_parameters=initialization_parameters,
    data_mapping=data_mapping
)

### Results Display
Display the evaluation results for each test case.

In [ ]:
pprint(results)